In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_market_calendars as mcal
import datetime as dt
import scipy as sp


import pymc3 as pm
from theano import tensor as tt
from theano import shared
import arviz as az
import seaborn as sns
az.style.use("arviz-darkgrid")

import os
import sys


from jupyterthemes import jtplot
jtplot.style(theme="monokai")

# Import data

In [2]:
os.chdir(sys.path[0]+"/raw_data")
os.listdir()

# Read in files and grab daily close prices
df = pd.DataFrame(data={"returns"})
for n, fname in enumerate(os.listdir()):
    if n == 0:
        ticker = fname.split(".csv")[0]
        df = pd.read_csv(fname, index_col="Date", parse_dates=True)[["Adj Close"]].rename(columns={"Adj Close" : ticker})
    else:
        ticker = fname.split(".csv")[0]
        temp_df = pd.read_csv(fname, index_col="Date", parse_dates=True)[["Adj Close"]].rename(columns={"Adj Close" : ticker})
        df = pd.merge(df, temp_df, left_index=True, right_index=True)
        
# Convert to daily log returns
df = np.log(df) - np.log(df.shift(1))
df = df.dropna()

# Input to model is each of these time series, so save
os.chdir(sys.path[0])
df.to_csv("log_returns.csv")

In [3]:
df.head()

,JNJ,LNG,MSFT,TSLA,V
Date,,,,,
2016-01-06,-0.005067,-0.037340,-0.018332,-0.019844,-0.013198
2016-01-07,-0.011723,-0.058048,-0.035402,-0.015598,-0.019858
2016-01-08,-0.010741,0.001143,0.003062,-0.021799,-0.012409
2016-01-11,-0.006029,-0.035469,-0.000574,-0.015041,0.014169
2016-01-12,0.006843,-0.009515,0.009136,0.010148,0.011299


In [4]:
data = df.values

In [5]:
nobs, ndim = data.shape

In [6]:
Sigma_a = np.random.randn(ndim, ndim)
Sigma_a = Sigma_a.T.dot(Sigma_a)
L_a = sp.linalg.cholesky(Sigma_a, lower=True)

In [8]:
data.shape

(1258, 5)

In [13]:
with pm.Model() as pd:
    # Model for volatility.
    vol_chol, vol_corr, vol_stds = pm.LKJCholeskyCov(
        "vol_chol", n=ndim, eta=2.0, sd_dist=pm.Exponential.dist(1.0), compute_corr=True
    )
    vol_cov = pm.Deterministic("vol_cov", vol_chol.dot(vol_chol.T))
    vols = pm.MvGaussianRandomWalk("alpha", mu=0, shape=(nobs, ndim), chol=vol_chol)
    vol_process = pm.Deterministic('vol_process', tt.exp(-2*vols))
    
    # Prior over correlation matrices
    corr_nu = pm.Uniform('corr_nu', 0, 5)
    C_triu = pm.LKJCorr('C_triu', corr_nu, ndim)
    
    # Combine vol_process and correlation matrices
    C = pm.Deterministic('C', tt.fill_diagonal(C_triu[np.zeros((ndim, ndim), dtype=np.int64)], 1.))
    
    # Prior for mean of observed returns
    mu = pm.Normal("mu", 0.0, 1.0, shape=ndim)
    
    # Tail parameter
    nu1 = pm.HalfNormal("nu_minus_2", sigma=1)
    nu2 = pm.Deterministic("nu", 2.0+nu1)
    
    for j in range(nobs):
        vol_array = tt.nlinalg.diag(vol_process[j,:]).sqrt()
        cov_array = tt.nlinalg.matrix_dot(vol_array, C, vol_array)
        r = pm.MvStudentT('obs_{}'.format(j), nu=nu2, mu=mu, cov=cov_array, observed=data[j,:])

In [12]:
with pm.Model() as pd:
    trace = pm.sample(1000, cores=1)

C:\Users\Jonathan\anaconda3\lib\site-packages\pymc3\sampling.py:465: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  warnings.warn(


ValueError: The model does not contain any free variables.

In [40]:
pm.MvGaussianRandomWalk.dist(mu=np.zeros((nobs, ndim)), shape=(nobs, ndim), chol=L_a).random()

AttributeError: 'MvGaussianRandomWalk' object has no attribute 'random'

In [41]:
from pymc3.distributions.timeseries import MvGaussianRandomWalk

In [44]:
with pm.Model():
    mu = np.array([1.0, 0.0])
    cov = np.array([[1.0, 0.0],
                    [0.0, 2.0]])
    sample = MvGaussianRandomWalk(mu, cov, shape=(10, 2)).random()

TypeError: 'NoneType' object is not callable

In [47]:
sample = MvGaussianRandomWalk.dist(mu, cov, shape=(10, 2)).random()

AttributeError: 'MvGaussianRandomWalk' object has no attribute 'random'

In [ ]:

            with pm.Model():
                mu = np.array([1.0, 0.0])
                cov = np.array([[1.0, 0.0],
                                [0.0, 2.0]])
                # draw one sample from a 2-dimensional Gaussian random walk with 10 timesteps
                sample = MvGaussianRandomWalk(mu, cov, shape=(10, 2)).random()
                # draw three samples from a 2-dimensional Gaussian random walk with 10 timesteps
                sample = MvGaussianRandomWalk(mu, cov, shape=(10, 2)).random(size=3)
                # draw four samples from a 2-dimensional Gaussian random walk with 10 timesteps,
                # indexed with a (2, 2) array
                sample = MvGaussianRandomWalk(mu, cov, shape=(10, 2)).random(size=(2, 2))
        """

In [ ]:
  #packed_L_a = pm.LKJCholeskyCov("packed_L_a", n=ndim, eta=2.0, sd_dist=pm.HalfCauchy.dist(2.5))
    #L_a = pm.expand_packed_triangular(ndim, packed_L_a)
    #a = pm.MvGaussianRandomWalk("alpha", mu=0, shape=(nobs, ndim), chol=L_a)
    #vol_process = pm.Deterministic('vol_process', tt.exp(-2*a))
    
    # Correlation in observed returns (independent of volatility)